In [5]:
# ! pip install transformers

In [1]:
# ! pip install torch

/bin/bash: pip: command not found


In [7]:
# ! pip install pandas

In [2]:
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizer, RobertaForMaskedLM, DataCollatorForLanguageModeling, pipeline, Trainer, TrainingArguments
import transformers
import torch
import pandas as pd
import os
import io
import json
from tqdm import tqdm

In [3]:
config = {
    'epochs': 2,
    'save_steps': 5000,
    'batch_size': 8,
    'checkpoint_dir': 'roberta_mlm_checkpoints/'
}

In [4]:
mlm_pipeline = pipeline(
    "fill-mask",
    model="roberta-base",
    tokenizer="roberta-base"
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [7]:
! unzip -q tokenized_papers.zip

In [8]:
! rm -r tokenized_papers/.DS_Store

In [9]:
tokenized_papers_path = 'tokenized_papers/'
class ACLAnthologyDataset(Dataset):

    def __init__(self):
        self.paper_names = []
        self.X = []
        for _, file in enumerate(tqdm(sorted(os.listdir(tokenized_papers_path), key = lambda x: int(x.strip('.json'))))):
            if _ >= 10000:
                break
            self.paper_names.append(file)
            x = ""
            with open(tokenized_papers_path + file, 'r', encoding='utf-8') as f:
                for line in json.loads(f.readlines()[0])['text'].strip(' ').strip('.').split('.'):
                    if len(line) + len(x) <= 800:
                        x += line
                    else:
                        self.X.append(tokenizer(x, truncation = True))
                        x = line

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx]

In [10]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
dataset = ACLAnthologyDataset()

 67%|███▉  | 10000/15004 [11:37<05:48, 14.34it/s]


In [12]:
training_args = TrainingArguments(
    output_dir=config['checkpoint_dir'],
    overwrite_output_dir=True,
    num_train_epochs=config['epochs'],
    per_device_train_batch_size=config['batch_size'],
    save_steps=config['save_steps'],
    save_total_limit=None,
    seed=1,
    disable_tqdm=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)
transformers.logging.set_verbosity_info()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
trainer.train(config['checkpoint_dir'] + 'roberta_checkpoint-85000')
# trainer.train()

trainer.save_model("./roberta-retrained")

Loading model from roberta_mlm_checkpoints/roberta_checkpoint-85000.
/home/ec2-user/.local/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 486253
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 121564
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 85000
  Will skip the first 1 epochs then the first 24218 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training o

  0%|          | 0/24218 [00:00<?, ?it/s]

Step,Training Loss
85500,1.400900
86000,1.392100
86500,1.380300
87000,1.387200
87500,1.366300
88000,1.388000
88500,1.375300
89000,1.370800
89500,1.382800
90000,1.378200


Saving model checkpoint to roberta_mlm_checkpoints/checkpoint-90000
Configuration saved in roberta_mlm_checkpoints/checkpoint-90000/config.json
Model weights saved in roberta_mlm_checkpoints/checkpoint-90000/pytorch_model.bin
Saving model checkpoint to roberta_mlm_checkpoints/checkpoint-95000
Configuration saved in roberta_mlm_checkpoints/checkpoint-95000/config.json
Model weights saved in roberta_mlm_checkpoints/checkpoint-95000/pytorch_model.bin
Saving model checkpoint to roberta_mlm_checkpoints/checkpoint-100000
Configuration saved in roberta_mlm_checkpoints/checkpoint-100000/config.json
Model weights saved in roberta_mlm_checkpoints/checkpoint-100000/pytorch_model.bin


KeyboardInterrupt: 